In [1]:
import sys
import logging
import os.path as p
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna

from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from bi_lstm import BiLSTM
from bert_embedding_model import BertEmbeddingModel
from debates_dataset import DebatesDataset
from optuna.trial import TrialState
from torchvision import transforms
# my transforms
from transforms import *

In [2]:
data = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_uw_ratio = 0
dataset_frac = 0.1
worthy_frac = 0.2
rs = 22

Function for loading data.

In [4]:
def load_data():
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
        ],
        'val': [
            p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_csv(dpaths[0], sep='\t', index_col=False)

        except Exception as e:
            print(e.args)
            exit()

Datasets and DataLoaders, takes trial as input to be able to suggest values for variables.

In [ ]:
def get_loaders(trial):
    # dev_df, test_df, train_df, val_df = data.values()
    subsets = {}
    for k, df in data.values():

        n_subset = len(df)*dataset_frac

        worthy_df = df.loc[df['label'] == 1]
        n_worthy = min(n_subset*worthy_frac, len(worthy_df))
        worthy_df = worthy_df.sample(n=n_worthy, random_state=rs)

        unworthy_df = df.loc[df['label'] == 0].sample(
            n=n_subset-n_worthy,
            random_state=rs
        )
        if k == 'train':
            train_uw_ratio = len(unworthy_df) / len(worthy_df)
        # sample(frac=1.0) -> shuffle
        subsets[k] = worthy_df.append(unworthy_df).sample(frac=1.0, random_state=rs, ignore_index=True)
    


    # transform_pipeline = transforms.Compose([
    #     Sum('pos', stopwords='wostop'),
    #     Sum('tag', stopwords='wostop'),
    #     ToBinary(6),
    #     ToTensor()
    # ])
    transform_pipeline = None

    train_dd = DebatesDataset(data=subsets['train'], transform=transform_pipeline)
    val_dd = DebatesDataset(data=subsets['val'], transform=transform_pipeline)
    test_dd = DebatesDataset(data=subsets['test'], transform=transform_pipeline)

    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    train_loader = DataLoader(train_dd, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dd, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dd, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader, test_loader

Model setup + training loop

In [ ]:
def objective(trial):
    train_loader, val_loader, test_loader = get_loaders(trial)

    # hyperparams opt
    pooling_strategy = trial.suggest_categorical('pooling_strategy', ['last_four', 'last_four_sum', 'second_last'])
    should_scale_emb = trial.suggest_categorical('should_scale_emb', [True, False])
    embedding_model = BertEmbeddingModel(
        device=device,
        pooling_strat=pooling_strategy,
        scale=should_scale_emb
    )
    
    dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.01)
    lstm_dropout = trial.suggest_float('lstm_dropout', 0.0, 0.3, step=0.05)
    hidden_dim = trial.suggest_categorical('hidden_dim', [64, 128, 256])
    model = BiLSTM(
        dropout=dropout,
        lstm_dropout=lstm_dropout,
        hidden_dim=hidden_dim,
        embedding_dim=embedding_model.dim,
        sent_level_feature_dim=0
    ).to(device)    

    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    opt_weight_decay = trial.suggest_float('optimizer_weigth_decay', 0, 0.1, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=opt_weight_decay)

    pos_weight = trial.suggest_categorical('pos_weight', [1.0, train_uw_ratio])
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))

    n_epochs = 10
    threshold = 0.5
    # training
    for epoch in range(n_epochs):
        model.train()
        for ids, sentences, labels, features in train_loader:
            labels = labels.float().to(device)

            optimizer.zero_grad()

            output = model(sentences)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

        model.eval()
        y_pred, y_true = [], []
        with torch.no_grad():
            for val_ids, val_sentences, val_labels, val_features in val_loader:
                val_labels = val_labels.float().to(device)
                pred = model(val_sentences)
                pred = torch.sigmoid(pred)

                pred = (pred > threshold).int()
                y_pred.extend(pred.tolist())
                y_true.extend(val_labels.tolist())

        cr = classification_report(y_true, y_pred, output_dict=True)
        recall_p = cr['1.0']['recall']

        trial.report(recall_p, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    "Done."
    return recall_p

In [5]:
load_data()

In [ ]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(
    study_name=f'bi-lstm_df{dataset_frac}_wf{worthy_frac}',
    sampler=optuna.samplers.RandomSampler(rs),
    pruner=optuna.pruners.MedianPruner(),
    direction='maximize'
)
study.optimize(objective, n_trials=100)

torch.save(study, os.path.join(EXP_DIR_PATH, 'bi-lstm', 'optimization', f'{study.study_name}.pkl'))